 <a name="Overview"></a>
# Overview

[Global Imports](#GlobImp)

[Global Classes](#GlobClas)

[Global Variables](#GlobVar)

[Global Functions](#GlobFunc)

[Mathematical Functions](#MathFunc)

[- Calculus](#MathFuncCalc)

[- Matrices](#MathFuncMat)

[Graphical Output](#GraphOut)

[Interactive Elements](#IntElem)

[- Calculations](#IntElemCalc)

[- Matrices](#IntElemMat)

[- 2D Graph](#IntElemGraph2D)

[Layout](#Layout)

<a name="GlobImp"></a>
# Global Imports

In [1]:
#%matplotlib inline
import os, sys, time, math
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import ipywidgets as widgets
from bqplot import pyplot as bqp
from bqplot.interacts import panzoom
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }.widget-htmlmath-content {width:700px !important;} </style>"))
plt.style.use('ggplot')
sp.init_printing()

 <a name="GlobClas"></a>
# Global Classes 
[Overview](#Overview)

In [2]:
# graphing classes

# line class
class is_line:
    def __init__(self, x_data, y_data, expr, x_sc, y_sc):
        self.x_data = x_data
        self.y_data = y_data
        self.x_sc = x_sc
        self.y_sc = y_sc
        self.expr = expr
            
# graph class
class is_graph:
    active = False
    position = 3
    x_min=-1
    x_max=1
    x_log=False
    y_log=False
    y_grid=False
    x_grid=False
    pi_bool=False
    axes0=False
    x_steps = 100
    def __init__(self, x_label='X', y_label='Y', title_label='Title', pi_bool=pi_bool,\
                 x_min=x_min, x_max=x_max, x_log=x_log, y_log=y_log, x_steps=x_steps,\
                 x_grid=x_grid, y_grid=y_grid, axes0=axes0):
        self.x_label = x_label
        self.y_label = y_label        
        self.title_label = title_label
        self.x_min = x_min
        self.x_max = x_max
        self.x_log = x_log
        self.y_log = y_log
        self.x_grid = x_grid
        self.y_grid = y_grid
        self.pi_bool = pi_bool
        self.axes0 = axes0
        selfx_steps = x_steps
        self.lines = []
        
    def addline(self, x_data, y_data, expr, x_sc, y_sc):
        self.lines.append(is_line(x_data, y_data))

<a name="GlobVar"></a>
# Global Variables
[Overview](#Overview)

In [3]:
#global variables
x, y, z, t = sp.symbols('x y z t')
xr, yr, zr = sp.symbols('xr yr zr', real=True)
xp, yp, zp = sp.symbols('xp yp zp', positive=True, nonzero=True)
f = sp.symbols('f', cls=sp.Function) 
ilim_max = sp.oo
ilim_min = -sp.oo
vardif = x
multdif = 1
xpoint = 0
utol = 100
ltol = -100

expr = x
expr_str = str(expr)
expr_array = []
subexpr = x

x_0 = x
x_1 = y
n_0 = 0
n_1 = 0

expr_0 = x
expr_1 = x
expr_2 = x
activeexpr = 0
activeexprlist = []

color_array = ['#0033cc', '#ff3300', '#33cc33', '#00ffcc', '#ff66cc', '#ff6600']
matrixinput = []
history_list = widgets.HTMLMath(
    value=r'Equations<br>',
    placeholder='Equations',
    layout = widgets.Layout(flex_wrap='nowrap', min_width='700px', max_height='450px', overflow_y='scroll'),
)
plot_array = []
fig1 = bqp.Figure(title='Graph 1')
fig2 = bqp.Figure(title='Graph 2')
fig3 = bqp.Figure(title='Graph 3')

outputfig = widgets.Output()

active_plot = 'fig1'
default_graph_settings = False

<a name="GlobFunc"></a>
# Global Functions 
[Overview](#Overview)

In [4]:
# makes a numerical computation of sympy equation
def npfy(x_, x_data, expr_):
    try:
        f = sp.lambdify(x_, expr_, 'numpy')
        y_data = f(x_data)
        y_data[y_data>utol] = np.inf
        y_data[y_data<ltol] = -np.inf
        return y_data
    except:
        f = sp.lambdify(x, x, 'numpy')
        y_data = f(x_data)
        return y_data
        
# changes variable for differentiation and integration
def set_vardif(self):
    global vardif
    try:
        vardif0 = sp.sympify(difint_input.value)
        vardif = vardif0
    except:
        error_out.value = 'Invalid value: ' + difint_input.value

# changes multiplicity variable for differentiation and integration
def setmultdif(self):
    global multdif
    try:
        multdif = int(multdif_input.value)
        if multdif <= 0:
            multdif = 1
            error_out.value = 'Positive integer nedded, got: ' + multdif_input.value
    except:
        multdif = 1
        error_out.value = 'Positive integer nedded, got: ' + multdif_input.value

# convert input to numerical values or sympy
def convertinput(input):
    try:
        if input.find('.') != -1:
            output = float(input)
        elif input.find(',') != -1:            
            output = float(input.replace(',', '.'))
        else:
            output = int(input)
        return output
    except:
        try:
            return sp.simpify(input)
        except:
            error_out.value = 'No valid input: ' + str(input)
            return False

# changes limits for integration
def setilim(self):
    global ilim_min
    global ilim_max
    ilim_min0 = convertinput(ilim_min_input.value)
    if ilim_min0:
        ilim_min = ilim_min0
    ilim_max0 = convertinput(ilim_max_input.value)
    if ilim_max0:
        ilim_max = ilim_max0

# changes tolerance limits for plotting
def settol(self):
    global ltol
    global utol
    ltol0 = convertinput(ltol_input.value)
    if ltol0:
        if ltol0 == -sp.oo:
            ltol = -np.inf
        else:
            ltol = ltol0
    utol0 = convertinput(utol_input.value)
    if utol0:
        if utol0 == sp.oo:
            utol = np.inf
        else:
            utol = utol0    

# set a point for expansion or limit
def setpoint(self):
    global xpoint
    xpoint0 = convertinput(xpoint_input.value)
    if xpoint0:
        xpoint = xpoint0

# adds the equation to history list
def add_history():
    global history_list
    history_list.value += str(len(expr_array)) + ' \(' + sp.latex(expr_array[-1]) + '\)' '<br>'
    eq_dropdown.options = [i + 1 for i, item in enumerate(expr_array)]
    return history_list

# outputs LaTeX string
def output_latex(self):
    latex_out.value = str(sp.latex(expr))

# saves history as latex to textfile
def save_history(self):
    global expr_array
    fname = time.strftime('%Y_%m_%d_%H_%M_%S', time.gmtime())
    path = 'output/equations_' + fname + '.txt'
    op = open(path, 'w')
    histstr = 'Equations\n\n'
    for i, hexpr in enumerate(expr_array):
        histstr += 'Eq ' + str(i + 1) + '\t' + sp.latex(hexpr) + '\n'
    history_list.value += str(len(expr_array)) + ' \(' + sp.latex(expr_array[-1]) + '\)' '<br>'
    eq_dropdown.options = [i + 1 for i, item in enumerate(expr_array)]
    op.write(histstr)
    return history_list

# clears the equations history
def clear_history(self):
    global history_list
    global expr_array
    history_list.value = r'Equations<br>'
    expr_array = []
    eq_dropdown.options = [0]

# sets choosen expresion active
def setactivegraph(self):
    global active_plot
    active_plot = active_dropdown.value
    if active_plot is 'fig1':
        plot = fig_array[0]
    elif active_plot is 'fig2':
        plot = fig_array[1]
    elif active_plot is 'fig3':
        plot = fig_array[2]
    if plot:
        global pi_bool_input
        global title_label_input
        global x_label_input
        global y_label_input
        global x_steps_input
        global x_min_input
        global x_max_input
        pi_bool_input.value = plot.pi_bool
        x_grid_input.value = plot.x_grid
        y_grid_input.value = plot.y_grid
        axes0_input.value = plot.axes0
        title_label_input.value = plot.title_label
        x_label_input.value = plot.x_label
        y_label_input.value = plot.y_label
        x_steps_input.value = str(plot.x_steps)
        x_min_input.value = str(plot.x_min)
        x_max_input.value = str(plot.x_max)

#sets choosen expresion active
def setactiveexpr():
    global activeexprlist
    global expr_str
    global expr
    for i, activeexpr0 in enumerate(activeexprlist):
        if i is activeexpr:
            activeexprlist[activeexpr][2].layout = widgets.Layout(border='1px solid #666666')
        else:
            activeexprlist[i][2].layout = widgets.Layout(border='1px dashed #999999')
    expr = activeexprlist[activeexpr][0]
    expr_str = str(expr)
    expr_input.value = expr_str
    
#sets up choosen expresion for substitution
def setsubexpr(num):
    global activeexprlist
    global expr
    expr = activeexprlist[num][0]
    for i, activeexpr in enumerate(activeexprlist):
        if i is num:
            activeexprlist[num][2].layout = widgets.Layout(border='1px dashed #666666')
        elif i is not activeexpr:
            activeexprlist[i][2].layout = widgets.Layout(border='1px dashed #999999')

#activates expressions
def activeexpr0(self):
    global activeexpr
    activeexpr = 0
    setactiveexpr()
def activeexpr1(self):
    global activeexpr
    activeexpr = 1
    setactiveexpr()
def activeexpr2(self):
    global activeexpr
    activeexpr = 2
    setactiveexpr()

#activates expressions for substitution
def subexpr0(self):
    setsubexpr(0)
def subexpr1(self):
    setsubexpr(1)
def subexpr2(self):
    setsubexpr(2)
    
#sets choosen expresion active
def setactive(self):
    global expr
    global expr_str
    pos = int(eq_dropdown.value) - 1
    if pos >= 0:
        expr = expr_array[pos]
        expr_str = str(expr)
        expr_input.value = expr_str

#sets choosen expresion active
def setnx(self):
    global x_0
    global n_0
    global x_1
    global n_1
    x_0_0 = convertinput(x0_input.value)
    if x_0_0:
        x_0 = x_0_0
    x_1_0 = convertinput(x1_input.value)
    if x_1_0:
        x_1 = x_1_0
    n_0_0 = convertinput(n0_input.value)
    if n_0_0:
        n_0 = n_0_0
    n_1_0 = convertinput(n1_input.value)
    if n_1_0:
        n_1 = n_1_0
        
#appends expression array and updates output
def expr_array_append(expr0):
    global expr_array
    global expr
    if len(expr_array) > 0:
        if expr_array[-1] is not expr0:
            expr = expr0
            expr_str = str(expr)
            expr_array.append(expr0)            
            activeexprlist[activeexpr][0] = expr
            activeexprlist[activeexpr][1].value = r'\(' + sp.latex(expr) + '\)'
            add_history()
    else:
        expr = expr0
        expr_str = str(expr)
        expr_array.append(expr)
        activeexprlist[activeexpr][0] = expr
        activeexprlist[activeexpr][1].value = r'\(' + sp.latex(expr) + '\)'
        add_history()

#reads and validates user input
def addexpr(change):
    global expr_str
    global expr
    if expr_input.value is not '' and expr_input.value is not None:        
        try:
            addexpr = sp.sympify(expr_input.value)
            expr = expr + addexpr
            expr_array_append(expr)
            return expr
        except:
            error_out.value = r'Invalid Expression!'
            return 'Invalid Expression!'    

#reads and validates user input
def readexpr(change):
    global expr_str
    global expr
    global expr_array
    if expr_input.value is not '' and expr_input.value is not None:        
        try:
            expr = sp.sympify(expr_input.value)
            expr_array_append(expr)
            return expr
        except:
            error_out.value = r'Invalid Expression!'
            return 'Invalid Expression!'
        
#reads and validates user input
def readmatrix():
    global matrixinput
    global expr_array
    mmax = 0
    nmax = 0
    mlist = []
    spok = True
    for m, m_row in enumerate(matrixinput):
        nlist = [] 
        hascell = False
        for n, m_cell in enumerate(m_row):
            if m_cell.value is not '' and m_cell.value is not None:
                spcell = convertinput(m_cell.value)
                if spcell:
                    nlist.append(spcell)
                    hascell = True
                else:
                    spok = False
                if (n + 1) > nmax:
                    nmax = n + 1
            else:
                nlist.append(0)
        if hascell:
            mmax = m + 1
        mlist.append(nlist)
    if spok:
        mlist = mlist[:mmax]
        for i in range(mmax):
            mlist[i] = mlist[i][:nmax]
        spmatrix = sp.Matrix(mlist)
        return spmatrix
    else:
        return False

<a name="MathFunc"></a>
# Mathematical Functions 
[Overview](#Overview)

In [5]:
# simplify equation
def simplify(self):
    sexpr = sp.simplify(expr)
    expr_array_append(sexpr)
    
# obtain the expression’s numeric value
def numval(self):
    nexpr = expr.n()
    expr_array_append(nexpr)

# solve equation
def solve(self):
    slexpr = sp.solve(expr, vardif)
    expr_array_append(slexpr)
    
# collect equation
def collect(self):
    colexpr = sp.collect(expr, vardif)
    expr_array_append(colexpr)
    
# factor equation
def factor(self):
    facexpr = sp.factor(expr)
    expr_array_append(facexpr)
    
# gets sum of an equation
def summation(self):
    sumeq = sp.summation(expr, [vardif, n_0, n_1])
    expr_array_append(sumeq)

# gets sum of an equation
def product(self):
    prodeq = sp.product(expr, [vardif, n_0, n_1])
    expr_array_append(prodeq)
    
# expands equation
def expand(self):
    eexpr = sp.expand(expr)
    expr_array_append(eexpr)
    
# expands complex equation
def expandcomp(self):
    eexpr = expr.expand(complex=True)
    expr_array_append(eexpr)
    
# rewrite complex equation
def rewritecomp(self):
    rexpr = expr.rewrite(sp.exp)
    expr_array_append(rexpr)
    
# get real part of expression 
def getre(self):
    reexpr = re(expr)
    expr_array_append(reexpr)
    
# get imaginary part of expression 
def getim(self):
    imexpr = im(expr)
    expr_array_append(imexpr)

# substitute a variable
def substitute(self):
    subexpr = sp.sympify(expr_input.value)
    exprsub = expr.subs(vardif, subexpr)
    expr_array_append(exprsub)

# substitute variables x_0 and x_1 by numeric value n_0 and n_1
def substitutexy(self):
    exprsub = expr.subs({x_0: n_0, x_1: n_1})
    expr_array_append(exprsub)


<a name="MathFuncCalc"></a>
## Calculus
[Overview](#Overview)

In [6]:
        
# derivatives
def derivative(self):
    exprdif = sp.diff(expr, vardif, multdif)
    expr_array_append(exprdif)
    
# find a limit
def limit(self):
    limexpr0 = sp.limit(expr, vardif, xpoint)
    expr_array_append(limexpr0)
    limexpr = limexpr0.doit()
    expr_array_append(limexpr)

# expand series
def expandseries(self):
    exprexp = expr.series(vardif, xpoint, multdif)
    expr_array_append(exprexp)
    
# integrals
def integrate(self):
    iexpr0 = sp.integrate(expr, vardif)
    expr_array_append(iexpr0)
    iexpr = iexpr0.doit()
    expr_array_append(iexpr)
    
def integratelim(self):
    iexpr0 = sp.integrate(expr, (vardif, ilim_min, ilim_max))
    expr_array_append(iexpr0)
    iexpr = iexpr0.doit()
    expr_array_append(iexpr)

<a name="MathFuncMat"></a>
## Matrices
[Overview](#Overview)

In [7]:
# transpose of matrix
def matrixtrans(self):
    matrix = expr.T
    if matrix:
        expr_array_append(matrix)

# norm matrix
def matrixnorm(self):
    try:
        matrix = expr.norm()
        if matrix:
            expr_array_append(matrix)
    except:
        error_out.value = 'Matrix can not be normalized!'        

# determinant matrix
def matrixdet(self):
    try:
        matrix = expr.det()
        if matrix:
            expr_array_append(matrix)
    except:
        error_out.value = 'No Determinant for Matrix!'

# invert matrix
def matrixinv(self):
    try:
        matrix = expr.inv()
        if matrix:
            expr_array_append(matrix)
    except:
        error_out.value = 'Matrix can not be inverted!'

        # dot product
def dotproduct(self):    
    try:
        matrix = readmatrix()
        if matrix:
            dotprod = expr.dot(matrix)
            expr_array_append(dotprod)
    except:
        error_out.value = 'Dot product can not be formed!'        

# cross product
def crossproduct(self):    
    try:
        matrix = readmatrix()
        if matrix:
            crossprod = expr.cross(matrix)
            expr_array_append(crossprod)
    except:
        error_out.value = 'Cross product can not be formed!'             

# get eigenvectors of expression
def eigenvectors(self):    
    try:
        matrix = expr.eigenvects()
        if matrix:
            expr_array_append(matrix)
    except:
        error_out.value = 'No Eigenvectors can be found!'        

# get eigenvalues of expression
def eigenvalues(self):    
    try:
        matrix = expr.eigenvals()
        if matrix:
            expr_array_append(matrix)
    except:
        error_out.value = 'No Eigenvalues can be found!'

# diagonalize expression
def diagonalize(self):    
    try:
        Q, L = expr.diagonalize()
        if L:
            expr_array_append(L)
    except:
        error_out.value = 'Matrix can not be diagonalized!'

# reads matrix and adds it to expression
def matrixadd(self):
    try:
        matrix = readmatrix()
        if matrix:
            mexpr = expr + matrix
            expr_array_append(mexpr)
    except:
        error_out.value = 'Matrix can not be be added'
        
# reads matrix and makes it expression
def matrixexpr(self):
    matrix = readmatrix()
    if matrix:
        expr_array_append(matrix)
        
# reads matrix and substitutes vardif by it
def matrixsub(self):
    matrix = readmatrix()
    if matrix:
        exprsub = expr.subs(vardif, matrix)
        expr_array_append(exprsub)

<a name="GraphOut"></a>
# Graphical Output
[Overview](#Overview)

In [8]:
# adds a line to a given plot
def addline(fig, plot, expr0):
    if plot.x_log:
        x_data = np.geomspace(plot.x_min, plot.x_max, plot.x_steps)
    else:
        x_data =  np.linspace(plot.x_min, plot.x_max, plot.x_steps)
    y_data = npfy(vardif, x_data, expr0)
    x_sc = bqp.LinearScale(min=plot.x_min, max=plot.x_max)
    y_sc = bqp.LinearScale()
    newline = is_line(x_data, y_data, expr0, x_sc, y_sc)
    line0 = bqp.Lines(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc})
    plot.lines.append(newline)
    fig.marks = fig.marks + [line0]
    return fig, plot

# make a new plot
def makeplot():
    if default_graph_settings:
        dgs = default_graph_settings
        newplot0 = is_graph(dgs.x_label, dgs.y_label, dgs.title_label, \
                 dgs.x_min, dgs.x_max, dgs.x_log, dgs.y_log, dgs.x_steps)
    else:
        newplot0 = is_graph()
    return newplot0

# get figure and plot
def setplot(plot0):
    global fig_array
    if active_plot is 'fig1':
        fig_array[0] = plot0
    elif active_plot is 'fig2':
        fig_array[1] = plot0
    elif active_plot is 'fig3':
        fig_array[3] = plot0

# get figure and plot
def getfigplot():
    global fig_array
    if active_plot is 'fig1':
        global fig1
        fig = fig1
        plot = fig_array[0]
    elif active_plot is 'fig2':
        global fig2
        fig = fig2
        plot = fig_array[1]
    elif active_plot is 'fig3':
        global fig3
        fig = fig3
        plot = fig_array[2]
    return fig, plot

# inserts a new line into the plot
def plotline(self):
    fig, plot = getfigplot()
    if plot:
        fig, plot = addline(fig, plot, expr)
        setplot(plot)
        updategraph('value')
    else:
        print(active_plot, pos)

# saves figure
def savefig(self):
    fig, plot = getfigplot()
    fig.save_png()

# clears all figures from plot
def clearfig(self):
    fig, plot = getfigplot()
    pos = eq_dropdown.value - 1
    fig.marks = []
    plot.lines = []
    updategraph('value')

# makes mathplotlib figure for saving
def mplgraph(self):
    global outputfig
    fig, plot0 = getfigplot()
    linedata = []
    ymin = 0
    ymax = 0
    if plot0.axes0:
        plt.axhline(y=0, color='k')
        plt.axvline(x=0, color='k')
    if plot0.x_grid or plot0.y_grid:
        plt.grid(True)
    for line in plot0.lines:
        plt.plot(line.x_data, line.y_data)
        ymin0 = np.min(line.y_data)
        ymax0 = np.max(line.y_data)
        if (ymin0 < ymin or ymin is 0):
            if float('-inf') < float(ymin0) < float('inf'):
                ymin = ymin0
            else:
                ymin = ltol
        if ymax0 > ymax:
            if float('-inf') < float(ymax0) < float('inf'):
                ymax = ymax0
            else:
                ymax = utol
    plt.title(plot0.title_label)
    plt.xlabel(plot0.x_label)    
    plt.ylabel(plot0.y_label)
    plt.axis([plot0.x_min, plot0.x_max, ymin - abs(ymin)*0.05, ymax + abs(ymax)*0.05])
    with outputfig:
        clear_output()
        plt.show()

# save figure
def savemplgraph(ending):
    global outputfig
    fname = time.strftime('%Y_%m_%d_%H_%M_%S', time.gmtime())
    path = 'output/graph_' + fname + ending
    fig, plot0 = getfigplot()
    linedata = []
    ymin = 0
    ymax = 0
    if plot0.axes0:
        plt.axhline(y=0, color='k')
        plt.axvline(x=0, color='k')
    if plot0.x_grid or plot0.y_grid:
        plt.grid(True)
    for line in plot0.lines:
        plt.plot(line.x_data, line.y_data)
        ymin0 = np.min(line.y_data)
        ymax0 = np.max(line.y_data)
        if (ymin0 < ymin or ymin is 0):
            if float('-inf') < float(ymin0) < float('inf'):
                ymin = ymin0
            else:
                ymin = ltol
        if ymax0 > ymax:
            if float('-inf') < float(ymax0) < float('inf'):
                ymax = ymax0
            else:
                ymax = utol
    plt.title(plot0.title_label)
    plt.xlabel(plot0.x_label)
    plt.ylabel(plot0.y_label)
    plt.axis([plot0.x_min, plot0.x_max, ymin - abs(ymin)*0.05, ymax + abs(ymax)*0.05])
    with outputfig:
        plt.savefig(path,  bbox_inches='tight')
        plt.close()
    
#save as svg
def savesvg(self):
    savemplgraph('.svg')
#save as svg
def savepng(self):
    savemplgraph('.png')

# updates plot when changes are done
def updategraph(change):
    global fig_array
    fig, plot = getfigplot()
    if plot:
        plot.title_label = title_label_input.value
        plot.x_label = x_label_input.value
        plot.y_label = y_label_input.value
        plot.x_grid = x_grid_input.value
        plot.y_grid = y_grid_input.value
        plot.axes0 = axes0_input.value
        try:
            plot.x_steps = int(x_steps_input.value)
        except:
            plot.x_steps = plot.x_steps
        try:
            plot.x_min = float(x_min_input.value)
        except:
            plot.x_min = plot.x_min
        try:
            plot.x_max = float(x_max_input.value)
        except:
            plot.x_max = plot.x_max
        if plot.x_log:
            x_sc = bqp.LogScale(min=plot.x_min, max=plot.x_max)
            x_data = np.geomspace(plot.x_min, plot.x_max, plot.x_steps)
        else:
            x_sc = bqp.LinearScale(min=plot.x_min, max=plot.x_max)
            x_data =  np.linspace(plot.x_min, plot.x_max, plot.x_steps)
        if plot.y_log:
            y_sc = bqp.LogScale()
        else:
            y_sc = bqp.LinearScale()
        ax_x = bqp.Axis(label=plot.x_label, scale=x_sc, grid_lines='none', grid_color='#999999')
        ax_y = bqp.Axis(label=plot.y_label, scale=y_sc, orientation='vertical', grid_lines='none', grid_color='#999999')        
        fig.marks = []
        ymin = 0
        ymax = 0
        for line in enumerate(plot.lines):
            y_data = npfy(vardif, x_data, line[1].expr)
            ymin0 = np.min(y_data)
            ymax0 = np.max(y_data)
            if ymin0 < ymin or ymin is 0:
                ymin = ymin0
            if ymax0 > ymax:
                ymax = ymax0
            line0 = bqp.Lines(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc}, colors=[color_array[line[0] % len(color_array)]])
            fig.marks = fig.marks + [line0]
        fig.axes = [ax_x, ax_y]
        if plot.x_grid:
            ax_x.grid_lines = 'solid'
        if plot.y_grid:
            ax_y.grid_lines = 'solid'
        if plot.axes0:
            lineax0 = bqp.Lines(x=[plot.x_min, plot.x_max], y=[0, 0], scales={'x': x_sc, 'y': y_sc}, colors=['#666666'])
            if ymin < ltol:
                ymin = ltol
            if ymax > utol:
                ymax = utol 
            lineax1 = bqp.Lines(x=[0, 0], y=[ymin, ymax], scales={'x': x_sc, 'y': y_sc}, colors=['#666666'])
            fig.marks = fig.marks + [lineax0, lineax1]
        default_graph_settings = plot
        fig.title = plot.title_label
        setplot(plot)


<a name="IntElem"></a>
# Interactive Elements 
[Overview](#Overview)

In [9]:
# styles for interactive elements
numberinputlayout = widgets.Layout(width='100px')
smallnumberinputlayout = widgets.Layout(width='65px', height='30px', margin='1px', padding='1px')
mclayout = widgets.Layout(width='25px', height='25px', margin='1px', padding='1px')

tinybtnlayout = widgets.Layout(width='25px')
smallbtnlayout = widgets.Layout(width='50px')
bigbtnlayout = widgets.Layout(width='100px')

descrstyle = {'description_width': 'initial'}


<a name="IntElemCalc"></a>
## Calculation 
[Overview](#Overview)

In [10]:
# interactive calculation elements
send_btn = widgets.Button(
    description=u'\u2397',
    layout=tinybtnlayout,
    tooltip='Use Equation'
)
add_btn = widgets.Button(
    description=u'+ \u2397',
    layout=smallbtnlayout,
    tooltip='Add Equation'
)
setactive_btn = widgets.Button(
    description=u'\u2BA7',
    layout=tinybtnlayout,
    tooltip='Use selected Equation',
)
active_dropdown = widgets.Dropdown(
    options=['fig1', 'fig2', 'fig3'],
    value='fig1',
    style=descrstyle,
    description='Use Graph:'
)

xpoint_input = widgets.Text(
    value='0',
    layout=numberinputlayout,
    style=descrstyle,
    description='Xval:'
)

#text elements
expr_input = widgets.Textarea(value=expr_str)
expr_label = widgets.Label(
    layout=widgets.Layout(width="100px", margin='2px', padding='2px'),
    value=u'Expression \u2BA7'
)
eq_dropdown = widgets.Dropdown(
    options=[1],
    value=1,
    layout=widgets.Layout(width="75px", margin='2px', padding='2px'),
)

x0_input = widgets.Text(
    value='x',
    layout=smallnumberinputlayout,
    style=descrstyle,
    description=u'x\u2080'
)
n0_input = widgets.Text(
    value='0',
    layout=smallnumberinputlayout,
    style=descrstyle,
    description=u'n\u2080'
)
x1_input = widgets.Text(
    value='y',
    layout=smallnumberinputlayout,
    style=descrstyle,
    description=u'x\u2081'
)
n1_input = widgets.Text(
    value='0',
    layout=smallnumberinputlayout,
    style=descrstyle,
    description=u'n\u2081'
)

# buttons

save_btn = widgets.Button(
    description='Save',
    layout=smallbtnlayout,
    tooltip='Save Fig as PNG'
)

plotline_btn = widgets.Button(
    description='Plot',
    layout=smallbtnlayout,
    tooltip='Plot Line',
)
newplot_btn = widgets.Button(
    description='New Plot',
    tooltip='New Plot',
)

clearfig_btn = widgets.Button(
    description='clear',
    layout=smallbtnlayout,
    tooltip='clear choosen Figure',
)
clearhist_btn = widgets.Button(
    description='clear',
    layout=smallbtnlayout,
    tooltip='clear History',
)

#output

sub0_btn = widgets.Button( description=u'\u2398',  tooltip='Substitute into equation 1', layout=tinybtnlayout)
sub1_btn = widgets.Button( description=u'\u2398',  tooltip='Substitute into equation 2', layout=tinybtnlayout)
sub2_btn = widgets.Button( description=u'\u2398',  tooltip='Substitute into equation 3', layout=tinybtnlayout)

acteq0_btn = widgets.Button( description=u'\u2397',  tooltip='activate equation 1', layout=tinybtnlayout)
acteq1_btn = widgets.Button( description=u'\u2397',  tooltip='activate equation 2', layout=tinybtnlayout)
acteq2_btn = widgets.Button( description=u'\u2397',  tooltip='activate equation 3', layout=tinybtnlayout)

expr_0_box = widgets.VBox(layout=widgets.Layout(border='1px solid #666666', width='auto', height='auto', flex='1 1 auto'))
expr_1_box = widgets.VBox(layout=widgets.Layout(border='1px dashed #999999', width='auto', height='auto', flex='1 1 auto'))
expr_2_box = widgets.VBox(layout=widgets.Layout(border='1px dashed #999999', width='auto', height='auto', flex='1 1 auto'))

sub0_btn.on_click(subexpr0)
sub1_btn.on_click(subexpr1)
sub2_btn.on_click(subexpr2)

acteq0_btn.on_click(activeexpr0)
acteq1_btn.on_click(activeexpr1)
acteq2_btn.on_click(activeexpr2)

eq_0_label = widgets.Label(value='Eq 0:', layout=smallbtnlayout)
eq_1_label = widgets.Label(value='Eq 1:', layout=smallbtnlayout)
eq_2_label = widgets.Label(value='Eq 2:', layout=smallbtnlayout)

eq_out0 = widgets.Label(value='')
eq_out1 = widgets.Label(value='')
eq_out2 = widgets.Label(value='')

latex_out = widgets.Label(value='')

error_out = widgets.Label(value='')

activeexprlist = [[expr_0, eq_out0, expr_0_box], [expr_1, eq_out1, expr_1_box], [expr_2, eq_out2, expr_2_box]]
    

<a name="IntElemExpr"></a>
### Expression
[Overview](#Overview)

In [11]:
# buttons

n_btn = widgets.Button(
    description='Num',
    layout=smallbtnlayout,
    tooltip='Get numerical value'
)
solv_btn = widgets.Button(
    description='Solv',
    layout=smallbtnlayout,
    tooltip='Solve Equation for d'
)
simp_btn = widgets.Button(
    description='Simp',
    layout=smallbtnlayout,
    tooltip='Simplify Equation',
)
expand_btn = widgets.Button(
    description='Exp',
    layout=smallbtnlayout,
    tooltip='Expand Equation',
)
fac_btn = widgets.Button(
    description='Fact',
    layout=smallbtnlayout,
    tooltip='Factor Equation',
)
collect_btn = widgets.Button(
    description='Col',
    layout=smallbtnlayout,
    tooltip='Collect Equation',
)


expandseries_btn = widgets.Button(
    description='Exp S',
    layout=widgets.Layout(width='75px'),
    tooltip='Expand Series Equation',
)
lim_btn = widgets.Button(
    description='lim',
    layout=smallbtnlayout,
    tooltip='Limit at point',
)
sum_btn = widgets.Button(
    description=u'\u2211\u207f\u00b9\u2099\u2092',
    layout=widgets.Layout(width='75px'),
    tooltip='Summation',
)
prod_btn = widgets.Button(
    description=u'\u2210\u207f\u00b9\u2099\u2092',
    layout=widgets.Layout(width='75px'),
    tooltip='Product',
)

im_btn = widgets.Button(
    description=u'\u2111',
    layout=tinybtnlayout,
    tooltip='Imaginary part',
)
re_btn = widgets.Button(
    description=u'\u211c',
    layout=tinybtnlayout,
    tooltip='Real part',
)
expandcomp_btn = widgets.Button(
    description=u'Exp \u2102',
    layout=widgets.Layout(width='75px'),
    tooltip='Expand Complex'
)
rewritecomp_btn = widgets.Button(
    description=u'RW \u2102',
    layout=widgets.Layout(width='75px'),
    tooltip='Rewrite Complex'
)

subs_btn = widgets.Button(
    description=u'\u2398',
    layout=tinybtnlayout,
    tooltip='Substitute active Equation by Input',
)
subsxy_btn = widgets.Button(
    description=u'\u2398 x x',
    layout=tinybtnlayout,
    tooltip='Substitute x and x by n and n',
)

<a name="IntElemCalculus"></a>
### Calculus
[Overview](#Overview)

In [12]:
difint_input = widgets.Text(
    value='x',
    layout=smallnumberinputlayout,
    style=descrstyle,
    description='d'
)
ilim_max_input = widgets.Text(
    value='oo',
    layout=numberinputlayout,
    style=descrstyle,
    description='Lmax:'
)
ilim_min_input = widgets.Text(
    value='-oo',
    layout=numberinputlayout,
    style=descrstyle,
    description='Lmin:'
)
multdif_input = widgets.Text(
    value='1',
    layout=numberinputlayout,
    style=descrstyle,
    description='Mult:'
)

dif_btn = widgets.Button(
    description='dif',
    layout=smallbtnlayout,
    tooltip='differentiate',
)
int_btn = widgets.Button(
    description=u'\u222b',
    layout=smallbtnlayout,
    tooltip='integrate',
)
intlim_btn = widgets.Button(
    description=u'\u222b\u00b9\u2092',
    layout=smallbtnlayout,
    tooltip='integrate within limits'
)

<a name="IntElemMat"></a>
### Matrices
[Overview](#Overview)

In [13]:
# Vectors and Matrices

m_0_0 = widgets.Text(value='', layout=mclayout)
m_0_1 = widgets.Text(value='', layout=mclayout)
m_0_2 = widgets.Text(value='', layout=mclayout)
m_0_3 = widgets.Text(value='', layout=mclayout)
m_0_4 = widgets.Text(value='', layout=mclayout)

m_0 = [m_0_0, m_0_1, m_0_2, m_0_3, m_0_4]

m_1_0 = widgets.Text(value='', layout=mclayout)
m_1_1 = widgets.Text(value='', layout=mclayout)
m_1_2 = widgets.Text(value='', layout=mclayout)
m_1_3 = widgets.Text(value='', layout=mclayout)
m_1_4 = widgets.Text(value='', layout=mclayout)

m_1 = [m_1_0, m_1_1, m_1_2, m_1_3, m_1_4]

m_2_0 = widgets.Text(value='', layout=mclayout)
m_2_1 = widgets.Text(value='', layout=mclayout)
m_2_2 = widgets.Text(value='', layout=mclayout)
m_2_3 = widgets.Text(value='', layout=mclayout)
m_2_4 = widgets.Text(value='', layout=mclayout)

m_2 = [m_2_0, m_2_1, m_2_2, m_2_3, m_2_4]

m_3_0 = widgets.Text(value='', layout=mclayout)
m_3_1 = widgets.Text(value='', layout=mclayout)
m_3_2 = widgets.Text(value='', layout=mclayout)
m_3_3 = widgets.Text(value='', layout=mclayout)
m_3_4 = widgets.Text(value='', layout=mclayout)

m_3 = [m_3_0, m_3_1, m_3_2, m_3_3, m_3_4]

m_4_0 = widgets.Text(value='', layout=mclayout)
m_4_1 = widgets.Text(value='', layout=mclayout)
m_4_2 = widgets.Text(value='', layout=mclayout)
m_4_3 = widgets.Text(value='', layout=mclayout)
m_4_4 = widgets.Text(value='', layout=mclayout)

m_4 = [m_4_0, m_4_1, m_4_2, m_4_3, m_4_4]

matrixinput = [m_0, m_1, m_2, m_3, m_4]

readmatrix_btn = widgets.Button(
    description=u'\u2397',
    layout=tinybtnlayout,
    tooltip='Read Matrix'
)
addmatrix_btn = widgets.Button(
    description=u'\u2397+',
    layout=tinybtnlayout,
    tooltip='Add Matrix'
)
normmatrix_btn = widgets.Button(
    description='Nrm',
    layout=smallbtnlayout,
    tooltip='Normalize Matrix'
)
detmatrix_btn = widgets.Button(
    description='det',
    layout=smallbtnlayout,
    tooltip='Determinant'
)
subsmat_btn = widgets.Button(
    description=u'\u2398',
    layout=tinybtnlayout,
    tooltip='Substitute active Equation by Matrix',
)
invmat_btn = widgets.Button(
    description=u'A\u207a\u00b9',
    layout=tinybtnlayout,
    tooltip='Invert Matrix',
)
transmat_btn = widgets.Button(
    description='T',
    layout=tinybtnlayout,
    tooltip='Transpose Matrix'
)
dotprod_btn = widgets.Button(
    description=u'\u2299',
    layout=tinybtnlayout,
    tooltip='Dot product',
)
crossprod_btn = widgets.Button(
    description=u'\u2297',
    layout=tinybtnlayout,
    tooltip='Cross product',
)
dia_btn = widgets.Button(
    description=u'\u22f1',
    layout=tinybtnlayout,
    tooltip='Diagonalize matrix',
)
eigenval_btn = widgets.Button(
    description='Eval',
    layout=smallbtnlayout,
    tooltip='Eigenvalues',
)
eigenvec_btn = widgets.Button(
    description='Evec',
    layout=smallbtnlayout,
    tooltip='Eigenvectors',
)

readmatrix_btn.on_click(matrixexpr)
addmatrix_btn.on_click(matrixadd)
normmatrix_btn.on_click(matrixnorm)
detmatrix_btn.on_click(matrixdet)
subsmat_btn.on_click(matrixsub)
invmat_btn.on_click(matrixinv)
transmat_btn.on_click(matrixtrans)
dotprod_btn.on_click(dotproduct)
crossprod_btn.on_click(crossproduct)
dia_btn.on_click(eigenvalues)
eigenvec_btn.on_click(eigenvectors)
eigenval_btn.on_click(eigenvalues)

In [14]:
setactive_btn.on_click(setactive)
send_btn.on_click(readexpr)
add_btn.on_click(addexpr)
multdif_input.observe(setmultdif, 'value')
xpoint_input.observe(setpoint, 'value')
save_btn.on_click(savefig)
plotline_btn.on_click(plotline)
clearfig_btn.on_click(clearfig)
clearhist_btn.on_click(clear_history)
active_dropdown.observe(setactivegraph, 'value')

x0_input.observe(setnx, 'value')
n0_input.observe(setnx, 'value')
x1_input.observe(setnx, 'value')
n1_input.observe(setnx, 'value')

expand_btn.on_click(expand)
simp_btn.on_click(simplify)
solv_btn.on_click(solve)
n_btn.on_click(numval)
collect_btn.on_click(collect)
fac_btn.on_click(factor)
sum_btn.on_click(summation)
prod_btn.on_click(product)
subs_btn.on_click(substitute)
subsxy_btn.on_click(substitutexy)

expandcomp_btn.on_click(expandcomp)
rewritecomp_btn.on_click(rewritecomp)
re_btn.on_click(getre)
im_btn.on_click(getim)

expandseries_btn.on_click(expandseries)
dif_btn.on_click(derivative)
int_btn.on_click(integrate)
intlim_btn.on_click(integratelim)
lim_btn.on_click(limit)

difint_input.observe(set_vardif, 'value')
ilim_min_input.observe(setilim, 'value')
ilim_max_input.observe(setilim, 'value')


<a name="IntElemGraph2D"></a>
## 2D Graphs
[Overview](#Overview)

In [15]:
# interactive graph elements

title_label_input = widgets.Text(
    value='Title',
    description='Title:'
)
x_label_input = widgets.Text(
    value='X',
    description='X-Label:',
    disabled=False)
y_label_input = widgets.Text(
    value='Y',
    description='Y-Label:'
)
x_steps_input = widgets.Text(
    value='100',
    layout=numberinputlayout,
    style=descrstyle,
    description='data points:'
)
x_min_input = widgets.Text(
    value='-1',
    layout=numberinputlayout,
    style=descrstyle,
    description='x min:'
)
x_max_input = widgets.Text(
    value='1',
    layout=numberinputlayout,
    style=descrstyle,
    description='x max:'
)
ltol_input = widgets.Text(
    value='-100',
    layout=numberinputlayout,
    style=descrstyle,
    description='y min:'
)
utol_input = widgets.Text(
    value='100',
    layout=numberinputlayout,
    style=descrstyle,
    description='y max:'
)
pi_bool_input = widgets.Checkbox(
    value=False,
    description='Multiples of Pi'
)
x_grid_input = widgets.Checkbox(
    value=False,
    description='X Grid'
)
y_grid_input = widgets.Checkbox(
    value=False,
    description='Y Grid'
)
axes0_input = widgets.Checkbox(
    value=False,
    description='Axes'
)
savetex_btn = widgets.Button(
    description='TeX',
    layout=smallbtnlayout,
    tooltip='Save ',
)
tex_btn = widgets.Button(
    description='TeX',
    layout=smallbtnlayout,
    tooltip='LaTeX String',
)
opfig_btn = widgets.Button(
    description='MPL',
    layout=smallbtnlayout,
    tooltip='Output Graph',
)
savesvg_btn = widgets.Button(
    description='SVG',
    layout=smallbtnlayout,
    tooltip='Save Fig as SVG',
)
savepng_btn = widgets.Button(
    description='PNG',
    layout=smallbtnlayout,
    tooltip='Save Fig as PNG',
)

savetex_btn.on_click(save_history)
tex_btn.on_click(output_latex)
opfig_btn.on_click(mplgraph)
savesvg_btn.on_click(savesvg)
savepng_btn.on_click(savepng)
title_label_input.observe(updategraph, 'value')
x_label_input.observe(updategraph, 'value')
y_label_input.observe(updategraph, 'value')
x_steps_input.observe(updategraph, 'value')
x_min_input.observe(updategraph, 'value')
x_max_input.observe(updategraph, 'value')
pi_bool_input.observe(updategraph, 'value')
x_grid_input.observe(updategraph, 'value')
y_grid_input.observe(updategraph, 'value')
axes0_input.observe(updategraph, 'value')
utol_input.observe(settol, 'value')
ltol_input.observe(settol, 'value')

<a name="Layout"></a>
# Layout
[Overview](#Overview)

In [16]:
# graphs
fig_array = []
for i in range(3):
    newplot = makeplot()
    fig_array.append(newplot)
    
# history area
eqchoice = widgets.HBox([clearhist_btn, active_dropdown, clearfig_btn, savetex_btn, plotline_btn, save_btn])
    
# vector and matrix input

matrixbox = widgets.VBox(layout= widgets.Layout(width='150px'))
matrixarray = []
for m_row in matrixinput:
    matrixrow = widgets.HBox(layout= widgets.Layout(height='30px'))
    matrixrow.children = [m_cell for m_cell in m_row]
    matrixarray.append(matrixrow)

matrixbox.children = matrixarray

matrixop_hbox0 = widgets.HBox([normmatrix_btn, invmat_btn, transmat_btn])
matrixop_hbox1 = widgets.HBox([dotprod_btn, crossprod_btn, addmatrix_btn, detmatrix_btn])
matrixop_hbox2 = widgets.HBox([eigenvec_btn, eigenval_btn, dia_btn])
matrixop_hbox3 = widgets.HBox([subsmat_btn, readmatrix_btn])
matrixop_vbox = widgets.VBox([matrixop_hbox0, matrixop_hbox1, matrixop_hbox2, matrixop_hbox3])

# expression output

eq0_hbox = widgets.HBox([eq_0_label, acteq0_btn, sub0_btn])
eq1_hbox = widgets.HBox([eq_1_label, acteq1_btn, sub1_btn])
eq2_hbox = widgets.HBox([eq_2_label, acteq2_btn, sub2_btn])

expr_0_box.children = [eq0_hbox, activeexprlist[0][1]]
expr_1_box.children = [eq1_hbox, activeexprlist[1][1]]
expr_2_box.children = [eq2_hbox, activeexprlist[2][1]]

# expression input
expr_hbox0 = widgets.HBox([expr_label, eq_dropdown, setactive_btn, send_btn, add_btn])
expr_hbox1 = widgets.HBox([x0_input, n0_input, x1_input, n1_input, difint_input])
expr_vbox = widgets.VBox([expr_hbox0, widgets.HBox([expr_input, subs_btn]), expr_hbox1])

# operations 
op_hbox0 = widgets.HBox([simp_btn, expand_btn, collect_btn, fac_btn])
op_hbox1 = widgets.HBox([solv_btn, n_btn])
op_hbox2 = widgets.HBox([rewritecomp_btn, expandcomp_btn, re_btn, im_btn])
op_hbox3 = widgets.HBox([subsxy_btn, prod_btn, sum_btn])
op_vbox = widgets.VBox([op_hbox0, op_hbox1, op_hbox2, op_hbox3])

calculus_box = widgets.HBox([xpoint_input, lim_btn, expandseries_btn,  multdif_input, dif_btn, ilim_min_input, ilim_max_input, int_btn, intlim_btn]) 

input_box = widgets.HBox([matrixbox, matrixop_vbox, expr_vbox, op_vbox])

calculation_box = widgets.VBox([input_box, calculus_box, error_out])
axes_box = widgets.VBox([widgets.HBox([x_label_input, x_min_input, x_max_input, x_steps_input]),\
                         widgets.HBox([y_label_input, utol_input, ltol_input]),\
                         widgets.HBox([title_label_input, x_grid_input, y_grid_input, axes0_input])])
input_tab = widgets.Tab([calculation_box, axes_box])
input_tab.set_title(0, 'Calculation')
input_tab.set_title(1, '2D Graph Settings')
left_box = widgets.VBox(layout= widgets.Layout(width='50%'))
left_box.children = [history_list, eqchoice, widgets.HBox([expr_0_box, expr_1_box, expr_2_box]), input_tab]

tb1 = bqp.Toolbar(figure=fig1)
tb2 = bqp.Toolbar(figure=fig2)
tb3 = bqp.Toolbar(figure=fig3)
fig_vbox = widgets.VBox([fig2, tb2, fig3, tb3])
fig3d_vbox = widgets.VBox([])
vector_vbox = widgets.VBox([])
tex_output =  widgets.HBox([tex_btn, latex_out])
figoutcontroll = widgets.HBox([opfig_btn, savesvg_btn, savepng_btn])
output_vbox = widgets.VBox([tex_output, figoutcontroll, outputfig])

viz_tab = widgets.Tab([fig_vbox, fig3d_vbox, vector_vbox, output_vbox])
viz_tab.set_title(0, '2D Graphs')
viz_tab.set_title(1, '3D Graph')
viz_tab.set_title(2, 'Vectors')
viz_tab.set_title(3, 'Output')
viz_vbox = widgets.VBox(layout= widgets.Layout(width='50%'))
viz_vbox.children = [fig1, tb1, viz_tab]

gui = widgets.HBox(layout= widgets.Layout(width='100%'))
gui.children=[left_box, viz_vbox]
gui

HBox(children=(VBox(children=(HTMLMath(value='Equations<br>', layout=Layout(max_height='450px', min_width='700px', overflow_y='scroll'), placeholder='Equations'), HBox(children=(Button(description='clear', layout=Layout(width='50px'), style=ButtonStyle(), tooltip='clear History'), Dropdown(description='Use Graph:', options=('fig1', 'fig2', 'fig3'), style=DescriptionStyle(description_width='initial'), value='fig1'), Button(description='clear', layout=Layout(width='50px'), style=ButtonStyle(), tooltip='clear choosen Figure'), Button(description='TeX', layout=Layout(width='50px'), style=ButtonStyle(), tooltip='Save '), Button(description='Plot', layout=Layout(width='50px'), style=ButtonStyle(), tooltip='Plot Line'), Button(description='Save', layout=Layout(width='50px'), style=ButtonStyle(), tooltip='Save Fig as PNG'))), HBox(children=(VBox(children=(HBox(children=(Label(value='Eq 0:', layout=Layout(width='50px')), Button(description='⎗', layout=Layout(width='25px'), style=ButtonStyle(), tooltip='activate equation 1'), Button(description='⎘', layout=Layout(width='25px'), style=ButtonStyle(), tooltip='Substitute into equation 1'))), Label(value='')), layout=Layout(border='1px solid #666666', flex='1 1 auto', height='auto', width='auto')), VBox(children=(HBox(children=(Label(value='Eq 1:', layout=Layout(width='50px')), Button(description='⎗', layout=Layout(width='25px'), style=ButtonStyle(), tooltip='activate equation 2'), Button(description='⎘', layout=Layout(width='25px'), style=ButtonStyle(), tooltip='Substitute into equation 2'))), Label(value='')), layout=Layout(border='1px dashed #999999', flex='1 1 auto', height='auto', width='auto')), VBox(children=(HBox(children=(Label(value='Eq 2:', layout=Layout(width='50px')), Button(description='⎗', layout=Layout(width='25px'), style=ButtonStyle(), tooltip='activate equation 3'), Button(description='⎘', layout=Layout(width='25px'), style=ButtonStyle(), tooltip='Substitute into equation 3'))), Label(value='')), layout=Layout(border='1px dashed #999999', flex='1 1 auto', height='auto', width='auto')))), Tab(children=(VBox(children=(HBox(children=(VBox(children=(HBox(children=(Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px'))), layout=Layout(height='30px')), HBox(children=(Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px'))), layout=Layout(height='30px')), HBox(children=(Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px'))), layout=Layout(height='30px')), HBox(children=(Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', layout=Layout(height='25px', margin='1px', padding='1px', width='25px')), Text(value='', 